In [2]:
# !pip3 install torchtext==0.4

In [70]:
import math
import numpy as np
import pandas as pd
import pdb

import torch
import torch.nn.functional as F
import torch.distributions
import torch.optim as optim
from torch import nn 
from torch.autograd import Variable

import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe

In [71]:
class Model(torch.nn.Module):
    def __init__(self, batch_size, num_classes, mlp_out_size, vocab_size, embedding_length, weights, num_layers, hidden_size = 100, biDirectional = False):
        super(Model, self).__init__() 
        """
        Arguments
        ---------
        batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
        num_classes : 28 = (For full classification)
        hidden_sie : Size of the hidden_state of the LSTM   (// Later BiLSTM)
        vocab_size : Size of the vocabulary containing unique words
        embedding_length : Embeddding dimension of GloVe word embeddings
        weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
        --------

        """

        self.batch_size = batch_size
        self.num_classes = num_classes
        self.vocab_size = vocab_size
        self.embedding_length = embedding_length
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.mlp_out_size = mlp_out_size
        self.biDirectional = biDirectional

        self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
        self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
        
        self.lstm_layer = LSTM(self.batch_size, self.hidden_size, self.embedding_length, self.biDirectional, self.num_layers)

        if(self.biDirectional):
            self.mlp = MLP(self.hidden_size*2, self.mlp_out_size, self.num_classes)
#             self.FF = nn.Linear(self.hidden_size*2, num_classes)
        else:
            self.mlp = MLP(self.hidden_size, self.mlp_out_size, self.num_classes)
#             self.FF = nn.Linear(self.hidden_size, self.num_classes)

    def forward(self, input_sequence):
        input_ = self.word_embeddings(input_sequence)
        out_lstm, final_hidden_state = self.lstm_layer(input_)
        if self.biDirectional:
            final_hidden_state = final_hidden_state.view(self.num_layers, 2, input_.shape[0], self.hidden_size) # num_layer x num_dir x batch x hidden
            final_hidden_state = final_hidden_state[-1]
            final_hidden_state = final_hidden_state.transpose(0,1).reshape(input_.shape[0], self.hidden_size*2)
        else:
            final_hidden_state = final_hidden_state[-1]
        
        mlp_output = self.mlp(final_hidden_state)
#         ff_output = self.FF(mlp_output)
#         print("FF out size: ", ff_output.shape)
#         predictions = torch.softmax(mlp_output, dim = -1)
        return mlp_output

In [72]:
class LSTM(torch.nn.Module):
    """
        Arguments
        ---------
        batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
        hidden_size : Size of the hidden_state of the LSTM   (* Later BiLSTM, check dims for BiLSTM *)
        embedding_length : Embeddding dimension of GloVe word embeddings
        --------
    """
    def __init__(self, batch_size, hidden_size, embedding_length, biDirectional = False, num_layers = 2):

        super(LSTM, self).__init__()
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.embedding_length = embedding_length
        self.biDirectional= biDirectional
        self.num_layers = num_layers

        self.lstm = nn.LSTM(self.embedding_length, self.hidden_size, bidirectional = self.biDirectional, batch_first = True, num_layers = self.num_layers)   # Dropout  

    def forward(self, input_sequence, batch_size=None):
        out_lstm, (final_hidden_state, final_cell_state) = self.lstm(input_sequence)   # ouput dim: ( batch_size x seq_len x hidden_size )
        return out_lstm, final_hidden_state

In [73]:
# If want to add extra MLP Layer
class MLP(torch.nn.Module):
    def __init__(self, input_dim, output_dim, num_classes):
        super(MLP, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_classes = num_classes

        self.ff_1 = nn.Linear(self.input_dim, self.output_dim)
        self.relu = nn.ReLU()
        self.ff_2 = nn.Linear(self.output_dim, self.num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        out_1 = self.ff_1(x)
        out_relu = self.relu(out_1)
        out_2 = self.ff_2(out_relu)
#         out_sigmoid = self.sigmoid(out_2)

        return out_2

In [74]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)

In [111]:
def train_model(model, optim, train_iter, epoch, batch_size, num_classes):
    total_epoch_loss = 0
    total_epoch_acc = 0
    total_epoch_prec = 0
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        text = batch.text[0]
        target = batch.labels
        target = torch.autograd.Variable(target)
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        if (text.size()[0] is not batch_size): # One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()
        target = target - 2
        prediction = model(text)
#         print(prediction.shape)
#         print(prediction)
#         print(target.shape)
#         print(target)
        loss =  loss_fn(prediction, target.type_as(prediction))
#         if math.isnan(loss.item()):
#             print(prediction, target)
        score = torch.sigmoid(prediction).cpu()       
        predicted = torch.round(score)
        num_corrects = (predicted.data == target.data).sum()
#         x = torch.LongTensor(1)
#         print(type(target))
        num_correct_precision = torch.where(target.type(torch.LongTensor) == 1, predicted.type(torch.LongTensor), target.type(torch.LongTensor)).sum()
        total_precision = torch.where(target == 1, target, target).sum()
        total_ones = torch.where(predicted == 1, predicted, predicted).sum()
        print("Total correct ones:",total_precision)
        print("Total ones:",total_ones)
        print("***************************************")
        precision = 100.0 * (num_correct_precision/total_precision)
        acc = 100.0 * (num_corrects/(len(batch)*num_classes))
    
#         num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
#         acc = 100.0 * num_corrects/len(batch)
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        total_epoch_prec = precision.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter), total_epoch_prec/len(train_iter)

In [85]:
def eval_model(model, val_iter, num_classes):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            
            text = batch.text[0]
            if (text.size()[0] is not 16):
                continue
            target = batch.labels
            target = torch.autograd.Variable(target)
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            target = target - 2
            prediction = model(text)
            
            score = torch.sigmoid(prediction).cpu()       
            predicted = torch.round(score)
            num_corrects = (predicted.data == target.data).sum()
#             print("Test Prediction: ", prediction)

            loss =  loss_fn(prediction, target.type_as(prediction))
            
            if math.isnan(loss.item()):
                print(prediction, target)
            
#             num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * (num_corrects/(len(batch)*num_classes))
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()
            
    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)

<>:9: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:9: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-85-86b55f64506a>:9: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (text.size()[0] is not 16):


In [86]:
def vectorize(x):
    vec = np.zeros(28, dtype=int)
    labels = x.split(',')
    for l in labels:
        l = int(l)
        vec[l] = 1
    return vec
#     return torch.from_numpy(vec)

def load_data(batch_size= 16, embedding_length = 100):
    tokenizer = lambda x: x.split()
    TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True, include_lengths=True, batch_first=True, fix_length=30)
    LABELS = data.Field(tokenize=vectorize, batch_first=True)

    train, val, test = data.TabularDataset.splits(
      path='/Users/prakruti/Documents/GoEmotions-classification/data', train='train.tsv',
      validation='dev.tsv', test='test.tsv', format='tsv',
      fields=[('text', TEXT), ('labels', LABELS)])
    
    train_iter, val_iter, test_iter = data.BucketIterator.splits(
      (train, val, test), batch_sizes=(16, 16, 16),
      sort_key=lambda x: len(x.text), device=0)

    # build the vocabulary
    TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=embedding_length))
    LABELS.build_vocab(train)
#     print(LABELS.vocab.__dict__)

    word_embeddings = TEXT.vocab.vectors
    vocab_size = len(TEXT.vocab)

    return TEXT, vocab_size, word_embeddings, train_iter, val_iter, test_iter

In [112]:
TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter = load_data()

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [88]:
# next(iter(train_iter))

In [113]:
# Over-writing the loss function to simple cross entropy loss
loss_fn = nn.BCEWithLogitsLoss()
# loss_fn = nn.BCELoss()

learning_rate = 2e-4
batch_size = 16
embedding_length = 100
num_classes = 28
mlp_out_size = 32
weights = word_embeddings
hidden_size = 100
num_layers = 3

model = Model(batch_size, num_classes, mlp_out_size, vocab_size, embedding_length, weights, num_layers, hidden_size, biDirectional=False)
optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

In [114]:
# num_bad_epochs = 0
epoch = 0
least_loss = float('inf')
training_stats = pd.DataFrame(columns=['Epoch', 'Train_Loss', 'Train_Acc', 'Val_Loss', 'Val_Acc'])

while(True):
    train_loss, train_acc, train_prec = train_model(model, optim, train_iter, epoch, batch_size, num_classes)
    val_loss, val_acc = eval_model(model, valid_iter, num_classes) 
    print(f'Epoch: {epoch+1:02}')
    if val_loss < least_loss:
        least_loss = val_loss
        num_bad_epochs = 0
        print("*** Least validation loss")
        torch.save(model.state_dict(), "LSTM_Multi_Label")
    else:
        num_bad_epochs += 1
#     print(f'Epoch: {epoch+1:2}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%')
    print(f'Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Train prec: {train_prec:.2f}% ') 
    print(f'Val Loss: {val_loss:3f}, Val Acc: {val_acc:.2f}%')
    print("-------------")
    
    training_stats = training_stats.append(
        pd.Series([epoch+1, train_loss, train_acc, val_loss, val_acc], index=training_stats.columns), 
        ignore_index=True)
    if num_bad_epochs >= 10:
        break
        
    epoch += 1
    if epoch == 50:
        break

Total correct ones: tensor(19)
Total ones: tensor(224., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(21)
Total ones: tensor(224., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(20)
Total ones: tensor(208., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total ones: tensor(194., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total ones: tensor(179., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(20)
Total ones: tensor(160., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(161., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(117., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total one

Total correct ones: tensor(18)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(21)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(20)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(20)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total ones: tensor(0., gr

Total correct ones: tensor(21)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(20)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., gr

Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(21)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(16)
Total ones: tensor(0., gr

Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(20)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(22)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(16)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(22)
Total ones: tensor(0., gr

...

Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(22)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(16)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(20)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(21)
Total ones: tensor(0., gr

Total correct ones: tensor(18)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(23)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(20)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(20)
Total ones: tensor(0., gr

Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(21)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(24)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(20)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(20)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., gr

Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(16)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(17)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(19)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(21)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(16)
Total ones: tensor(0., grad_fn=<SumBackward0>)
***************************************
Total correct ones: tensor(18)
Total ones: tensor(0., gr

KeyboardInterrupt: 